In [55]:
val input = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
                    Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
                    Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
                    Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
                    Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"""
                   

In [56]:
fun parseSets(sets: String): List<MutableMap<String, Int>> {
    return sets.splitToSequence(";")
        .map { set ->
            set.splitToSequence(",")
                .fold(mutableMapOf<String, Int>()) { res, cubes ->
                    val (num, color) = cubes.trim().split(" ")
                    res[color] = num.toInt()
                    res
                }
        }.toList()
}


fun parseGame(game: String): Pair<Int, List<MutableMap<String, Int>>> {
    val (name, sets) = game.split(":")
    val (_, gameId) = name.split(" ")

    return Pair(gameId.toInt(), parseSets(sets))
}


val a = input.splitToSequence('\n').map { parseGame(it.trim()) }.toList()
println(a.toList())

[(1, [{blue=3, red=4}, {red=1, green=2, blue=6}, {green=2}]), (2, [{blue=1, green=2}, {green=3, blue=4, red=1}, {green=1, blue=1}]), (3, [{green=8, blue=6, red=20}, {blue=5, red=4, green=13}, {green=5, red=1}]), (4, [{green=1, red=3, blue=6}, {green=3, red=6}, {green=3, blue=15, red=14}]), (5, [{red=6, blue=1, green=3}, {blue=2, red=1, green=2}])]


In [73]:
val bag = mapOf("red" to 12, "green" to 13, "blue" to 14)

val validGames = a.filter { (gameId, gameSets) ->
    !gameSets.any { set -> set.entries.any { (color, count) -> bag.get(color)!! < count } }
}.map { (gameId, _) -> gameId }.sum()

println(validGames)

//val t = bag.values
//println(t)


8


In [75]:
val b = a.map { (id, sets) -> sets }
println(b.toList())

[[{blue=3, red=4}, {red=1, green=2, blue=6}, {green=2}], [{blue=1, green=2}, {green=3, blue=4, red=1}, {green=1, blue=1}], [{green=8, blue=6, red=20}, {blue=5, red=4, green=13}, {green=5, red=1}], [{green=1, red=3, blue=6}, {green=3, red=6}, {green=3, blue=15, red=14}], [{red=6, blue=1, green=3}, {blue=2, red=1, green=2}]]


In [95]:
import io.kotest.core.spec.style.featureSpec

val c = b.map { sets ->
    sets.fold(mutableMapOf<String, Int>()) { res, set ->
        set.entries.forEach {
            val (color, count) = it
            res[color] = res.get(color)?.let { max(it, count) } ?: count
        }
        res
    }
} 

in res:{} set:{blue=3, red=4}
out res:{blue=3, red=4}
in res:{blue=3, red=4} set:{red=1, green=2, blue=6}
out res:{blue=6, red=4, green=2}
in res:{blue=6, red=4, green=2} set:{green=2}
out res:{blue=6, red=4, green=2}
in res:{} set:{blue=1, green=2}
out res:{blue=1, green=2}
in res:{blue=1, green=2} set:{green=3, blue=4, red=1}
out res:{blue=4, green=3, red=1}
in res:{blue=4, green=3, red=1} set:{green=1, blue=1}
out res:{blue=4, green=3, red=1}
in res:{} set:{green=8, blue=6, red=20}
out res:{green=8, blue=6, red=20}
in res:{green=8, blue=6, red=20} set:{blue=5, red=4, green=13}
out res:{green=13, blue=6, red=20}
in res:{green=13, blue=6, red=20} set:{green=5, red=1}
out res:{green=13, blue=6, red=20}
in res:{} set:{green=1, red=3, blue=6}
out res:{green=1, red=3, blue=6}
in res:{green=1, red=3, blue=6} set:{green=3, red=6}
out res:{green=3, red=6, blue=6}
in res:{green=3, red=6, blue=6} set:{green=3, blue=15, red=14}
out res:{green=3, red=14, blue=15}
in res:{} set:{red=6, blue=1, gr

In [98]:
c.forEach { println("> ${it}") }

val d = c.map { it.values.reduce { acc, i -> acc * i } }.sum()

println(d)

> {blue=6, red=4, green=2}
> {blue=4, green=3, red=1}
> {green=13, blue=6, red=20}
> {green=3, red=14, blue=15}
> {red=6, blue=2, green=3}
2286


In [26]:
val input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
                    Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
                    Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
                    Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
                    Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
                    Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""


data class Card(val winningNumbers: List<Int>, val ownedNumbers: List<Int>) {

    companion object {
        fun of(input: String): Card {
            val (winningNbs, ownedNumbers) = line.substringAfter(":").split("|", limit = 2)
                .map { numbers -> numbers.trim().split(Regex("\\s+")).map { it.toInt() } }
            return Card(winningNbs, ownedNumbers)
        }
    }

    override fun toString(): String {
        return "$winningNumbers|$ownedNumbers"
    }

    fun points(): Int {
        var winning = winningNumbers.intersect(ownedNumbers).size
        return 1.shl(winning - 1)
    }
}

fun card_of(input: String): Card {
    val (winningNbs, ownedNumbers) = line.substringAfter(":").split("|", limit = 2)
        .map { numbers -> numbers.trim().split(Regex("\\s+")).map { it.toInt() } }
    return Card(winningNbs, ownedNumbers)
}


//    
//    
val line = "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"
var res = card_of(line)
println(res.points())
    
    

8


In [64]:
val input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
                    Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
                    Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
                    Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
                    Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
                    Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""


data class Card(val id: Int, val winningNumbers: List<Int>, val ownedNumbers: List<Int>, var count: Int = 1) {

    companion object {
        fun of(input: String): Card {
            var id = input.trim().substringAfter(" ").substringBefore(":").toInt()
            val (winningNbs, ownedNumbers) = input.substringAfter(":").split("|", limit = 2)
                .map { numbers -> numbers.trim().split(Regex("\\s+")).map { it.toInt() } }
            return Card(id, winningNbs, ownedNumbers)
        }
    }

    override fun toString(): String {
        return "$id:$winningNumbers|$ownedNumbers"
    }

//    fun setCount(count: Int) {
//        this.count = count
//    }

    fun winnings(): List<Int> {
        var winning = winningNumbers.intersect(ownedNumbers).size
        return if (winning > 0) (id + 1..id + winning).map { it } else listOf()
    }

    fun points(): Int {
        var winning = winningNumbers.intersect(ownedNumbers).size
        return 1.shl(winning - 1)
    }
}


val cards = input.splitToSequence('\n').map { Card.of(it) }.toMutableList()


cards.forEach { card ->
    val winnings = card.winnings()
//    println("$winnings winnings for ${card.id}")
    winnings.forEach { winning ->
//        println("current count for ${winning}: cards to ${cards.elementAt(winning - 1).count}")
//        println("adding ${card.count} cards to ${winning}")
        cards.elementAt(winning - 1).count = cards.elementAt(winning - 1).count + card.count
//        println("new count for ${winning} cards: ${cards.elementAt(winning - 1).count}")
    }
}

val res = cards.sumOf { it.count }
println(res)


//    
//    
//    val line = "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"
//    var res = card_of(line) 
//    println( res.points())
//    

[2, 3, 4, 5] winnings for 1
current count for 2: cards to 1
adding 1 cards to 2
new count for 2 cards: 2
current count for 3: cards to 1
adding 1 cards to 3
new count for 3 cards: 2
current count for 4: cards to 1
adding 1 cards to 4
new count for 4 cards: 2
current count for 5: cards to 1
adding 1 cards to 5
new count for 5 cards: 2
[3, 4] winnings for 2
current count for 3: cards to 2
adding 2 cards to 3
new count for 3 cards: 4
current count for 4: cards to 2
adding 2 cards to 4
new count for 4 cards: 4
[4, 5] winnings for 3
current count for 4: cards to 4
adding 4 cards to 4
new count for 4 cards: 8
current count for 5: cards to 2
adding 4 cards to 5
new count for 5 cards: 6
[5] winnings for 4
current count for 5: cards to 6
adding 8 cards to 5
new count for 5 cards: 14
[] winnings for 5
[] winnings for 6
30


In [66]:
val r = (1..0).toList()
println(r)


[]
